In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

from utils.preprocessing import get_texts
from utils.preprocessing import get_texts, stop_words

In [7]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")

In [8]:
df_esg_score["sector"].unique()

array(['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare',
       'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology',
       'Financial Services', 'Communication Services', 'Real Estate', nan],
      dtype=object)

In [9]:
sector = "Financial Services"
tickers = df_esg_score[df_esg_score["sector"] == sector]["Company"]
esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]

In [10]:
score = esgs["governanceScore"]

In [11]:
upper_score = np.quantile(score, 0.75)
lower_score = np.quantile(score, 0.25)

In [12]:
bad_companies = esgs[esgs["governanceScore"] > upper_score]["Company"].values
good_companies = esgs[esgs["governanceScore"] < lower_score]["Company"].values

In [13]:
good_companies

array(['WFC', 'USB', 'SIVB', 'AIG', 'RJF', 'CMA', 'C', 'AIZ', 'SCHW',
       'WRB', 'CINF', 'PGR', 'AFL', 'UNM', 'AMP', 'AJG'], dtype=object)

In [14]:
bad_companies

array(['STT', 'PNC', 'HBAN', 'AXP', 'TROW', 'RF', 'SYF', 'PYPL', 'V',
       'ICE', 'CME', 'WU', 'SPGI', 'MSCI', 'NDAQ', 'MCO'], dtype=object)

In [15]:
esgs[esgs["governanceScore"] > upper_score]

,Company,socialScore,governanceScore,environmentScore
34,WFC,16.15,15.64,1.31
66,USB,15.85,12.44,1.71
75,SIVB,12.92,14.11,2.11
99,AIG,10.81,14.71,1.87
106,RJF,12.94,12.30,1.92
109,CMA,13.13,12.21,1.69
137,C,10.83,12.64,1.75
153,AIZ,10.10,12.69,1.66
157,SCHW,10.16,11.93,2.33
159,WRB,7.59,14.67,2.04


In [16]:
bad_companies_score = esgs[esgs["governanceScore"] > upper_score]["governanceScore"].values
good_companies_score = esgs[esgs["governanceScore"] < lower_score]["governanceScore"].values

In [17]:
avg_bad = np.mean(bad_companies_score)
avg_good = np.mean(good_companies_score)
print(avg_bad, avg_good)

13.0375 6.81125


In [18]:
print(upper_score, lower_score)

11.8325 8.67


In [19]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))
good_cik = []
bad_cik = []
for ticker in good_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        good_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        good_cik.append('')

for ticker in bad_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        bad_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        bad_cik.append('')

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [199]:
all_cik = []
for ticker in tickers:    
    try:
        # for a given ticker, find its cik number through th ticker library
        all_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        all_cik.append('')


In [ ]:
get_texts(all_cik, tickers)

In [26]:
good_companies = list(good_companies) #[:13] + good_companies[14:]

In [34]:
good_cik

['0000072971',
 '0000036104',
 '0000719739',
 '0000005272',
 '0000720005',
 '0000028412',
 '0000831001',
 '0001267238',
 '0000316709',
 '0000011544',
 '0000020286',
 '0000080661',
 '0000004977',
 '0000005513',
 '0000820027',
 '0000354190']

In [35]:
ret_good = get_texts(good_cik[1:14] + good_cik[15:], good_companies[1:14] + good_companies[15:])

14it [00:08,  1.73it/s]


In [22]:
good_cik[18]

'0000354190'

In [ ]:
ret_good = get_texts(good_cik[15:], good_companies[15:])

In [36]:
ret_bad = get_texts(bad_cik, bad_companies)

16it [00:00, 17.86it/s]


In [ ]:
ret_bad = get_texts(bad_cik, bad_companies)

In [ ]:
ret_bad = get_texts(bad_cik, bad_companies)

In [37]:
good_docs = ret_good["docs"]
bad_docs = ret_bad["docs"]

In [39]:
cv = CountVectorizer(max_df=0.8, stop_words=stop_words, max_features=500, ngram_range=(1,3))
word_count_vector = cv.fit_transform(good_docs + bad_docs)

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [40]:
cv.toarray().sum(axis=0)

AttributeError: 'CountVectorizer' object has no attribute 'toarray'

In [41]:
feature_names = cv.get_feature_names()

In [157]:
good_docs_list = []
bad_docs_list = []
for doc in good_docs:
    li = doc.split(' ')
    good_docs_list.append(set(li))
for doc in bad_docs:
    li = doc.split(' ')
    bad_docs_list.append(set(li))

In [43]:
d = {"word": [], "good_score": [], "bad_score": [], "good_score_all": []
    , "bad_score_all": [], "good_nums": [], "bad_nums": []}

for word in feature_names[:100]:
    good_sum = bad_sum = good_num = bad_num = 0

    for i, doc_set in enumerate(good_docs):
        if word in doc_set:
            good_num += 1
            good_sum += good_companies_score[i]
    for i, doc_set in enumerate(bad_docs):
        if word in doc_set:
            bad_num += 1
            bad_sum += bad_companies_score[i]
    
    # print("word: {}".format(word))
    d["word"].append(word) 
    
    if good_num:
        d["good_score"].append(good_sum / good_num)
    else:
        d["good_score"].append(0)
    if bad_num:
        d["bad_score"].append(bad_sum / bad_num)
    else:
        d["bad_score"].append(0)

    d["good_score_all"].append(good_sum / len(good_docs))
    d["bad_score_all"].append(bad_sum / len(bad_docs))

    d["good_nums"].append(good_num)
    d["bad_nums"].append(bad_num)


In [44]:
df = pd.DataFrame(data=d)


In [45]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,good_nums,bad_nums
0,accident,7.065556,13.488333,4.542143,5.058125,9,6
1,accident year,7.331667,0.000000,3.142143,0.000000,6,0
2,account asset,7.475000,13.543333,3.203571,2.539375,6,3
3,accounting guidance,6.990000,13.020000,6.990000,7.323750,14,9
4,acquisition cost,6.841818,13.612500,5.375714,6.806250,11,8


In [46]:
df["diff"] = abs(df["good_nums"] - df["bad_nums"])

In [48]:
df.sort_values("diff", ascending=False).head(60)

,word,good_score,bad_score,good_score_all,bad_score_all,good_nums,bad_nums,diff
6,adjustment expense,7.044545,12.690000,5.535000,0.793125,11,1,10
70,ceded,6.742222,14.710000,4.334286,0.919375,9,1,8
60,capital ratio,7.080000,13.625000,6.068571,5.109375,12,6,6
1,accident year,7.331667,0.000000,3.142143,0.000000,6,0,6
65,casualty,7.028889,13.603333,4.518571,2.550625,9,3,6
79,citi,6.703750,13.013571,3.830714,11.386875,8,14,6
88,combined ratio,7.331667,12.440000,3.142143,0.777500,6,1,5
29,appreciation,7.047500,12.872857,6.040714,5.631875,12,7,5
45,balance beginning,6.910833,13.075714,5.923571,5.720625,12,7,5
21,allowance loan loss,7.347000,13.528000,5.247857,4.227500,10,5,5


In [166]:
len(df)

100

In [165]:
df.to_csv("energy_good_vs_bad_uni_bi_tri.csv")

In [4]:
df = pd.read_csv("energy_good_vs_bad_uni_bi_tri.csv", index_col=0)

In [5]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,good_nums,bad_nums,diff
0,accompanying note consolidated,9.773333,23.7600,4.886667,3.960000,3,1,2
1,accompanying note integral,10.557500,21.6950,7.038333,7.231667,4,2,2
2,acmp,0.000000,21.9800,0.000000,3.663333,0,1,1
3,activity cash flow,10.975000,21.0660,7.316667,17.555000,4,5,1
4,adjusted ebitda,0.000000,22.2825,0.000000,14.855000,0,4,4


In [237]:
comp = pd.read_csv("data/sp500_component_stocks.csv")


In [241]:
comp[comp['A'] == "C"]["Agilent Technologies Inc."]

79    Citigroup Inc.
Name: Agilent Technologies Inc., dtype: object

In [140]:

comp[comp['A'].isin(good_companies)]["Agilent Technologies Inc."]

44                   Apache Corporation
111         Cabot Oil & Gas Corporation
128                 Chevron Corporation
163                  EOG Resources Inc.
322            Marathon Oil Corporation
358    Occidental Petroleum Corporation
Name: Agilent Technologies Inc., dtype: object

In [141]:
comp[comp['A'].isin(bad_companies)]["Agilent Technologies Inc."]

215            Halliburton Company
274      Kinder Morgan Inc Class P
343    National Oilwell Varco Inc.
354                     ONEOK Inc.
411                Schlumberger NV
487        Williams Companies Inc.
Name: Agilent Technologies Inc., dtype: object